# Title

description

* link to [video](https://www.youtube.com/watch?v=dLCGAuiJ1U0) for assignment overview
* analysis 1 con be done in conjustion with the What-if analysis with python module

## Preliminaries

In [9]:
# To auto-reload modules in jupyter notebook (so that changes in files *.py doesn't require manual reloading):
# https://stackoverflow.com/questions/5364050/reloading-submodules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [13]:
%matplotlib inline

## Create Model

First we will need to import the things we need from the whatif module

In [17]:
from whatif import Model
from whatif import get_sim_results_df

In [31]:
class SingleProductSPF(Model):
    """HW2 Model"""
    def __init__(self, fixed_cost=5000, var_cost=100, selling_price=115, 
                 spf_constant=4900, spf_linear=-35, spf_quadratic=0.06):
        self.fixed_cost = fixed_cost
        self.var_cost = var_cost
        self.selling_price = selling_price
        self.spf_constant = spf_constant
        self.spf_linear = spf_linear
        self.spf_quadratic = spf_quadratic

    def demand(self):
        """Computing Demand"""
        demand = 
        return demand

In [33]:
model = SingleProductSPF()
print(model)

{'fixed_cost': 5000, 'var_cost': 100, 'selling_price': 115, 'spf_constant': 4900, 'spf_linear': -35, 'spf_quadratic': 0.06}


In [39]:
model.data_table?

Signature: model.data_table(scenario_inputs, outputs)
Docstring:
Create n-inputs by m-outputs data table.

Parameters
----------
scenario_inputs : dict of str to sequence
    Keys are input variable names and values are sequence of values for each scenario for this variable.

outputs : list of str
    List of output variable names

Returns
-------
results_df : pandas DataFrame
    Contains values of all outputs for every combination of scenario inputs
File:      c:\users\22nic\onedrive\documents\ou-summer24\hw-files\hw2_excel_withpython\whatif.py
Type:      method